In [2]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


# Descargar data

In [3]:
# Lista de índices con sus respectivos tickers en Yahoo Finance
indexes = [
    {'id': 'CSPXL.XC', 'name': 'iShares Core S&P 500 UCITS ETF'},
    {'id': 'SPY', 'name': 'US Equity'},
    {'id': 'IEUR', 'name': 'Europe Equity'},
    {'id': 'EWJ', 'name': 'Japan Equity'},
    {'id': 'MCHI', 'name': 'China Equity'},
    {'id': 'EMXC', 'name': 'Ex-China Emerging Markets Equity'},
]

#indexes = [
#    {'id': 'BTC-USD', 'name': 'Bitcoin'},
#    {'id': 'ETH-USD', 'name': 'Ethereum'},
#    #{'id': 'XRP-USD', 'name': 'Ripple'},
#    #{'id': 'USDT-USD', 'name': 'Tether USDt'},
#    {'id': 'TRX-USD', 'name': 'Tron'},
#    #{'id': 'IMV', 'name': 'Argentina Equity'}
#]

# Definir rango de fechas
start_date = "2020-01-01"
end_date = "2025-12-31"


# Descargar datos históricos y construir el DataFrame consolidado
dfs = []
for index in indexes:
    ticker = index['id']
    print(f'Descargando {ticker}')
    stock_data = yf.download(ticker, start=start_date, multi_level_index=False,progress=False) #,end=end_date
    # Seleccionar solo la columna 'Close' y renombrarla
    stock_data = stock_data[['Close']].rename(columns={'Close': f'{ticker}_close'})
    
    # Agregar el DataFrame a la lista
    dfs.append(stock_data)

# Unir todos los DataFrames en uno solo usando el índice (fecha)
df = pd.concat(dfs, axis=1)

# Convertir el índice en una columna 'datetime'
df.reset_index(inplace=True)
df.rename(columns={'Date': 'datetime'}, inplace=True)
df['day_of_week_num'] = df['datetime'].dt.weekday
df.dropna(inplace=True)



Descargando CSPXL.XC
YF.download() has changed argument auto_adjust default to True
Descargando SPY
Descargando IEUR
Descargando EWJ
Descargando MCHI
Descargando EMXC


# Procesar data

In [4]:
for index in indexes:
    ticker = index['id']
    start_price = df[f'{ticker}_close'].iloc[0]
    df[f'{ticker}_pct'] = (df[f'{ticker}_close'] - start_price) / start_price * 100

In [5]:
# Crear una lista de las columnas *_pct
pct_columns = [col for col in df.columns if '_pct' in col]

# Crear un gráfico con todas las columnas *_pct
fig = px.line(df, x='datetime', y=pct_columns, title="Cambio Porcentual Diario de los Índices", labels={'datetime': 'Fecha'})
fig.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Cambio Porcentual (%)",
    legend_title="Índices",
    template="plotly_dark"  # Puedes elegir el tema que prefieras
)

# Mostrar el gráfico
fig.show()

## Estrategia de rebalanceo

In [6]:
capital_inicial = 10000
asset_perc = 1 / len(indexes)  # Repartir el capital 
perc_gap = asset_perc/10


# Establecer el capital inicial por activo 
capital_por_asset = capital_inicial * asset_perc

# Inicializar el capital para cada estrategia
capital_hold = []
capital_rebal = []
capital_usd = []
capital_assets = []

#Stock por asset
qty_hold  = {}
qty_rebal = {}
qty_hold['USD'] = 0.0
qty_rebal['USD'] = 0.0

# Inicializar valores para el capital invertido
for index in indexes:
    ticker = index['id']
    qty_hold[ticker]  = capital_por_asset/df.iloc[0][f'{ticker}_close']
    qty_rebal[ticker] = capital_por_asset/df.iloc[0][f'{ticker}_close']
    


k=0
for i,row in df.iterrows():
    hold = 0
    rebal = 0
    rebal_adj = 0
    
    at_start = {}
    at_end = {}
    for index in indexes:
        ticker = index['id']
        price = row[f'{ticker}_close']
        hold += qty_hold[ticker]*price
        rebal += qty_rebal[ticker]*price

    #Retirar Ganancia cuando el capital supera el capital inicial
    if rebal > capital_inicial*1.02:
        to_usd = rebal-capital_inicial
        rebal -= to_usd 
        qty_rebal['USD'] += to_usd

    if row['day_of_week_num'] == 2: #Procesa una ves a la semana 0=Lunes
        for index in indexes:
            ticker = index['id']
            price = row[f'{ticker}_close']
            
            actual_cap = qty_rebal[ticker]*price
            actual_perc = actual_cap/rebal
            correct_cap = rebal*asset_perc
            adjust_cap = actual_cap-correct_cap
            adjust_qty = adjust_cap/price
            qty_rebal[ticker] -= adjust_qty
            rebal_adj += qty_rebal[ticker]*price

    else:
        rebal_adj = rebal

    capital_hold.append(hold)
    capital_rebal.append(rebal_adj+qty_rebal['USD'])
    capital_usd.append(qty_rebal['USD'])
    capital_assets.append(rebal_adj)
    



# Crear el gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['datetime'], y=capital_hold, mode='lines', name='Hold'))
fig.add_trace(go.Scatter(x=df['datetime'], y=capital_rebal, mode='lines', name='Rebalanceo'))
fig.add_trace(go.Scatter(x=df['datetime'], y=capital_usd, mode='lines', name='USD en resguardo'))

# Personalizar el gráfico
fig.update_layout(
    title='Comparación de Capital: Hold vs Rebalanceo',
    xaxis_title='Fecha',
    yaxis_title='Capital ($)',
    template='plotly_dark',
    legend_title='Estrategia'
)
resultado = ((capital_rebal[-1]/capital_hold[-1])-1)*100
print(f'Rebal vs Hold : {resultado:.2f}%')


fig.show()



Rebal vs Hold : 35.63%
